# EyeFormer

## Imports

In [24]:
!pip install -qq ruamel.yaml==0.17.21

In [2]:
import argparse
import os
import ruamel.yaml as yaml
import numpy as np
import random
import time
import datetime
import json
from pathlib import Path
import csv

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn
import torch.distributed as dist

Clone Eyeformer & Create the environment

In [3]:
!pip install pytorch==1.7.1 torchvision==0.8.2 torchaudio==0.7.2 cudatoolkit==11.0
!pip install opencv-python==4.5.3.56 Pillow einops multimatch-gaze
!pip install transformers==4.8.1

ERROR: Could not find a version that satisfies the requirement pytorch==1.7.1 (from versions: 0.1.2, 1.0.2)
ERROR: No matching distribution found for pytorch==1.7.1
  Using cached opencv-python-4.5.3.56.tar.gz (89.2 MB)
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
  Using cached transformers-4.8.1-py3-none-any.whl.metadata (48 kB)
  Using cached huggingface_hub-0.0.12-py3-none-any.whl.metadata (5.6 kB)
  Using cached sacremoses-0.1.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached tokenizers-0.10.3.tar.gz (2

In [7]:
!pip install timm==0.4.9

In [13]:
!git clone https://github.com/YueJiang-nj/EyeFormer-UIST2024.git

Cloning into 'EyeFormer-UIST2024'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 95 (delta 23), reused 88 (delta 18), pack-reused 0
Receiving objects: 100% (95/95), 330.14 KiB | 9.71 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [14]:
%cd EyeFormer-UIST2024

/content/EyeFormer-UIST2024


### **NOTE**

Please change the `eval_image_root` string in `configs/Tracking.yaml` to your eval image dir

In [15]:
import gdown
url = 'https://drive.google.com/drive/folders/1eae63VRBfQ9XAYj3k2q2N9xDtLmSqQg_'
gdown.download_folder(url, quiet=True)
!mv scanpath_prediction_population_rl weights

In [16]:
from models.model_tracking import TrackingTransformer
from models.vit import interpolate_pos_embed

import utils
from dataset import create_dataset, create_sampler, create_loader
import csv

## Inference

In [17]:
def test(model, data_loader, tokenizer, device, output_dir, config):
    # train
    model.eval()

    metric_logger = utils.MetricLogger(delimiter="  ")
    header = 'Testing:'

    image_names = []
    results = []
    widths = []
    heights = []
    # user_ids = []
    for i, (image, image_name, width, height) in enumerate(metric_logger.log_every(data_loader, 1, header)):
        image = image.to(device, non_blocking=True)
        # user_id = user_id.to(device, non_blocking=True)

        coord = model.inference(image)
        coord = coord.cpu().numpy().tolist()

        width = width.numpy().tolist()
        height = height.numpy().tolist()

        # user_id = user_id.cpu().numpy().tolist()

        image_names.extend(image_name)
        results.extend(coord)
        widths.extend(width)
        heights.extend(height)
        # user_ids.extend(user_id)

    with open(os.path.join(output_dir, 'predicted_result.csv'), 'w') as wfile:
        writer = csv.writer(wfile)
        writer.writerow(["image", "width", "height", "x", "y", "timestamp"])

        for image, width, height, coord in zip(image_names, widths, heights, results):

            for row in coord:
                x = row[0] * width
                y = row[1] * height
                t = row[2]
                # username = data_loader.dataset.id2user[user_id]
                writer.writerow([image, width, height,
                                x, y, t])

    return

In [18]:
def main(args, config):
    utils.init_distributed_mode(args)

    device = torch.device(args.device)

    # fix the seed for reproducibility
    seed = args.seed + utils.get_rank()
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    cudnn.benchmark = True

    #### Dataset ####
    print("Creating dataset")
    datasets = [create_dataset('inference', config)]

    if args.distributed:
        num_tasks = utils.get_world_size()
        global_rank = utils.get_rank()
        samplers = create_sampler(datasets, [True], num_tasks, global_rank)
    else:
        samplers = [None]

    data_loader = create_loader(datasets,
                                samplers,
                                batch_size=[config['batch_size_test']],
                                num_workers=[32],
                                is_trains=[False],
                                collate_fns=[None])[0]

    # tokenizer = BertTokenizer.from_pretrained(args.text_encoder)
    tokenizer = None

    #### Model ####
    print("Creating model")
    model = TrackingTransformer(config=config, init_deit=False)

    model = model.to(device)

    if args.checkpoint:
        checkpoint = torch.load(args.checkpoint, map_location='cpu')
        state_dict = checkpoint['model']

        msg = model.load_state_dict(state_dict)
        print('load checkpoint from %s' % args.checkpoint)
        print(msg)

    model_without_ddp = model

    print("Start testing")
    start_time = time.time()

    test(model, data_loader, tokenizer, device, args.output_dir, config)

    dist.barrier()

    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print('Testing time {}'.format(total_time_str))

In [19]:
class ARGS:
    config = './configs/Tracking.yaml'
    checkpoint = './weights/checkpoint_19.pth'
    resume = False
    output_dir = 'output/tracking_eval'
    text_encoder = 'bert-base-uncased'
    device ='cuda'
    seed = 42
    world_size = 1
    dist_url = 'env://'
    distributed = True

In [27]:
from ruamel.yaml import YAML

In [29]:
args = ARGS()

yaml = YAML(typ='rt')
config = yaml.load(open(args.config, 'rt'))

Path(args.output_dir).mkdir(parents=True, exist_ok=True)
yaml.dump(config, open(os.path.join(args.output_dir, 'config.yaml'), 'w'))
main(args, config)

Not using distributed mode
Creating dataset


FileNotFoundError: [Errno 2] No such file or directory: '/home/researcher/Documents/saliency_modeling_issues/UEyes_dataset/images'